# Tube rearrangement

Import the output file from the **tube extraction** stage:

In [1]:
from os import path, getcwd
from pcgvs.extraction import load_tubes_with_pandas

# Generating an OS safe path.
p = path.join(getcwd(), 'synopsis')
p = path.join(p, 'tubes')
p = path.join(p, 'exp')
p = path.join(p, 'tracks')
p = path.join(p, 'video-1-raw.txt')

dataframe = load_tubes_with_pandas(p)
dataframe.head(10)

,frame,tag,x,y,w,h
0,4,1,592,553,37,39
1,5,1,592,553,37,39
2,5,2,593,554,35,38
3,6,1,591,554,38,39
4,7,1,592,555,36,38
5,7,2,590,554,39,40
6,8,1,590,554,39,40
7,8,2,589,554,40,40
8,9,1,590,554,39,40
9,9,2,589,554,40,40


Transform the dataframe to a list of Tube objects (see specifications at `pcgvs.extraction.Tube`) 

In [2]:
from pcgvs.extraction import load_tubes_from_pandas_dataframe

tubes = load_tubes_from_pandas_dataframe(dataframe)

for tube in tubes:
    print(f'tube of object {tube.tag} starts at frame {tube.sframe} and end at frame {tube.eframe}')

tube of object 1 starts at frame 4 and end at frame 14
tube of object 2 starts at frame 5 and end at frame 61
tube of object 3 starts at frame 44 and end at frame 92
tube of object 7 starts at frame 58 and end at frame 64
tube of object 13 starts at frame 146 and end at frame 204
tube of object 16 starts at frame 152 and end at frame 163
tube of object 18 starts at frame 192 and end at frame 245
tube of object 22 starts at frame 203 and end at frame 208
tube of object 27 starts at frame 372 and end at frame 439
tube of object 28 starts at frame 372 and end at frame 435
tube of object 29 starts at frame 391 and end at frame 441
tube of object 36 starts at frame 433 and end at frame 438


We use the tube rearrangement algorithm from [He Et al.](https://www.sciencedirect.com/science/article/pii/S0925231216313406) along with some of the optimizations proposed in [Allegra Et al.](https://ieeexplore.ieee.org/document/8803795).

The first step of the algorithm consist in computing the relations between the tubes. The relation between two generic tubes $T_1$ and $T_2$ can be one of the following:

- Irrilevant relation (`None`): the tubes trajectories doesn't intersect
- Intersecting relation (`INT`): the tubes trajectories intersect in some frames
- Overlapping relation (`OVL`): the tubes trajectories overlap in a long sequence of frames

In order to distinguish the intersection and the overlapping, the paper specify a threshold set to 5 frames, meaning that if the frames overlaps for more than 5 consecutive frames, then it's an overlapping relation, otherwise it's an intersecting relation.

In [3]:
from pcgvs.aggregation.relations import RelationsMap

relations = RelationsMap(tubes)

print(relations)

100%|███████████████████████████████████████████████████████████████████████████████| 132/132 [00:00<00:00, 168.96it/s]

[1]:	(1)None	(2)OVL	(3)None	(7)None	(13)OVL	(16)OVL	(18)None	(22)None	(27)OVL	(28)OVL	(29)None	(36)None	
[2]:	(1)OVL	(2)None	(3)None	(7)OVL	(13)OVL	(16)OVL	(18)None	(22)OVL	(27)OVL	(28)OVL	(29)None	(36)OVL	
[3]:	(1)None	(2)None	(3)None	(7)None	(13)None	(16)None	(18)OVL	(22)None	(27)None	(28)None	(29)OVL	(36)None	
[7]:	(1)None	(2)OVL	(3)None	(7)None	(13)OVL	(16)None	(18)None	(22)OVL	(27)None	(28)OVL	(29)None	(36)OVL	
[13]:	(1)OVL	(2)OVL	(3)None	(7)OVL	(13)None	(16)OVL	(18)None	(22)OVL	(27)OVL	(28)OVL	(29)None	(36)OVL	
[16]:	(1)OVL	(2)OVL	(3)None	(7)None	(13)OVL	(16)None	(18)None	(22)None	(27)OVL	(28)OVL	(29)None	(36)None	
[18]:	(1)None	(2)None	(3)OVL	(7)None	(13)None	(16)None	(18)None	(22)None	(27)None	(28)None	(29)OVL	(36)None	
[22]:	(1)None	(2)OVL	(3)None	(7)OVL	(13)OVL	(16)None	(18)None	(22)None	(27)None	(28)OVL	(29)None	(36)OVL	
[27]:	(1)OVL	(2)OVL	(3)None	(7)None	(13)OVL	(16)OVL	(18)None	(22)None	(27)None	(28)OVL	(29)None	(36)INT	
[28]:	(1)OVL	(2)OVL	(3)None	(7)OVL	(13)OVL	(16)OVL	

From the relation map, we need to compute the Potential Collision Graph. This graph is composed by two types of nodes: 

- main-node (m-node): representing a tube
- sub-node (s-node): representing a potential collision between two tubes

The following is a summary of how the nodes are generated from the relations map: 

- We generate a m-node for each tube
- For each intersection between $T_1$ and $T_2$: 
    - we generate a s-node in $T_1$ containing the frame of intersection from $T_1$
    - we generate a s-node in $T_2$ containing the frame of intersection from $T_2$
- For each overlapping between $T_1$ and $T_2$:
    - we generate two s-node in $T_1$, one for the starting frame and one for the ending frame
    - we generate two s-node in $T_2$, one for the starting frame and one for the ending frame

After the node generation step, we need to wire the nodes with edges. The process is summarized as follows: 

- For each intersection: 
    - add an edge weighted 1 between the two s-nodes
- For each overlapping: 
    - add a directed edge between the starting and ending node weighted using $\Delta t$
    - add an edge between the starting nodes of $T_1$ and $T_2$ weighted -1
    - add an edge between the ending nodes of $T_1$ and $T_2$ weighted -1
    
> With $\Delta t$ we refer to the difference in frames between the end and the start of the collision

In [4]:
from pcgvs.aggregation.graph import PCG

pcg = PCG(tubes, relations)

We use the graph coloring technique from He Et al. paper, based on a mix between the DSATUR algorithm and the RFL algorithm, adapted for the L(q)-coloring problem. 

A brief description of the algorithm is presented here: 

1. Set the initial color $c=1$
2. Let $U$ be the list of the uncolored nodes
3. Compute the saturation values (see eq. (2) from He Et al.)
3. Order $U$ by saturation values
4. For each node $v \in U$ check if the coloring conditions are met (see Cond.1 and Cond.2 from He Et al.)
    1. If the conditions are met, color the node with $c$
    2. If the node is generated from an overlapping, color the directly connected node with $c \pm \Delta t$ (the sign is determined by the direction) 
5. Increase $c = c+1$ and repeat the process from step 2. until $U = \emptyset$

Check the saturation computation at `pcgvs.aggregation.coloring.SaturationCache`.

A suggested q value (q=3) for cars is proposed by the authors, so we set the parameter here:

In [5]:
q=3

In [6]:
from pcgvs.aggregation.coloring import color_graph

color_graph(pcg, q)

Coloring node 2-1-s to 1
Coloring node 2-1-e to 102
Coloring node 2-7-s to 1
Coloring node 2-7-e to 56
Coloring node 2-13-s to 1
Coloring node 2-13-e to 108
Coloring node 2-16-s to 1
Coloring node 2-16-e to 102
Coloring node 2-22-s to 1
Coloring node 2-22-e to 30
Coloring node 2-27-s to 1
Coloring node 2-27-e to 108
Coloring node 2-28-s to 1
Coloring node 2-28-e to 108
Coloring node 2-36-s to 1
Coloring node 2-36-e to 86
Coloring node 13-1-s to 1
Coloring node 13-1-e to 114
Coloring node 13-7-s to 1
Coloring node 13-7-e to 46
Coloring node 13-16-s to 1
Coloring node 13-16-e to 116
Coloring node 13-22-s to 1
Coloring node 13-22-e to 26
Coloring node 13-27-s to 1
Coloring node 13-27-e to 118
Coloring node 13-28-s to 1
Coloring node 13-28-e to 118
Coloring node 13-36-s to 1
Coloring node 13-36-e to 64
Coloring node 3-18-s to 1
Coloring node 3-18-e to 88
Coloring node 3-29-s to 1
Coloring node 3-29-e to 88
Coloring node 18-29-s to 1
Coloring node 18-29-e to 92
Coloring node 27-1-s to 1
Col

Then we use the *Condensation optimization* from Allegra Et al. in order to calculate the tubes starting time in the video synopsis: 

In [7]:
from pcgvs.aggregation.coloring import tubes_starting_time

starting_times = tubes_starting_time(pcg, q)
starting_times

{1: 1,
 2: 4,
 7: 7,
 13: 10,
 22: 13,
 27: 16,
 28: 19,
 16: 22,
 36: 25,
 3: 1,
 18: 4,
 29: 7}

Later we will use starting times to generate the synopsis. 

In [ ]:
import json

with open('./synopsis/starting_times.json', 'w') as ssfile:
    writablejson = { str(k):v for k, v in starting_times.items() }
    json.dump(writablejson, ssfile)